# A. Create object and initial analysis

- Authors: Marcos Malumbres & Agustín Sánchez-Belmonte
- Project: miR-203 controls developmental timing and early fate restriction during preimplantation embryogenesis
- Experiment: single cell RNAseq in early embryos (E3.5 and E4.5) in KO, KI and WT conditions.
- Part: A. Create object and initial analysis

The input of this notebook are the output of STARsolo analysis. 

From raw data of Gene:
- matrix.mtx
- barcodes.tsv
- features.tsv

From raw data of Velocito:
- unspliced.mtx
- spliced.mtx
- ambigouos.mtx

From filtered data of Gene:
- barcodes.tsv

With all these data we create anndata object.

### Content

0. Set up
1. Load data and create h5 file
2. Initial exploratory analysis
3. Filtering 
4. Normalization
5. Dimensionality reduction
6. Clustering


# 0. Set up

In [ ]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy import sparse
from bioinfokit import analys, visuz
import gseapy as gspy
import scvelo as scv
from matplotlib import rcParams
import anndata

In [ ]:
sc.set_figure_params(dpi=120, color_map='viridis')
sc.settings.verbosity = 3
sc.logging.print_header()
rcParams['figure.figsize'] = 4, 4

sc.set_figure_params(dpi_save=300)

In [ ]:
pal1 = ["lightblue", "deepskyblue", "dodgerblue", "navajowhite", "darkorange", "orangered"]
responders_order = [""]

ORIGIN = '/Users/asanchezb/Desktop/jgonzalezm_scGEX_230316/'
DATA = '/Users/asanchezb/Desktop/FELINE_ASB/DATA/'

DESKTOP = '/Users/asanchezb/Desktop/'
DESKTOP_scanpy = '/Users/asanchezb/Desktop/'
sc.settings.figdir = DESKTOP_scanpy
scv.set_figure_params()

### Function

This function is to create an anndata object from STARsolo output (.out) with gene and velocyto information

In [ ]:
def buildAnndataFromStarCurr(path):
    """Generate an anndata object from the STAR aligner output folder"""
    path=path
    # Load Read Counts
    X = sc.read_mtx(path+'Gene/raw/matrix.mtx')

    # Transpose counts matrix to have Cells as rows and Genes as cols as expected by AnnData objects
    X = X.X.transpose()

    # Load the 3 matrices containing Spliced, Unspliced and Ambigous reads
    mtxU = np.loadtxt(path+'Velocyto/raw/unspliced.mtx', skiprows=3, delimiter=' ')
    mtxS = np.loadtxt(path+'Velocyto/raw/spliced.mtx', skiprows=3, delimiter=' ')
    mtxA = np.loadtxt(path+'Velocyto/raw/ambiguous.mtx', skiprows=3, delimiter=' ')

    # Extract sparse matrix shape informations from the third row
    shapeU = np.loadtxt(path+'Velocyto/raw/unspliced.mtx', skiprows=2, max_rows = 1 ,delimiter=' ')[0:2].astype(int)
    shapeS = np.loadtxt(path+'Velocyto/raw/spliced.mtx', skiprows=2, max_rows = 1 ,delimiter=' ')[0:2].astype(int)
    shapeA = np.loadtxt(path+'Velocyto/raw/ambiguous.mtx', skiprows=2, max_rows = 1 ,delimiter=' ')[0:2].astype(int)

    # Read the sparse matrix with csr_matrix((data, (row_ind, col_ind)), shape=(M, N))
    # Subract -1 to rows and cols index because csr_matrix expects a 0 based index
    # Traspose counts matrix to have Cells as rows and Genes as cols as expected by AnnData objects

    spliced = sparse.csr_matrix((mtxS[:,2], (mtxS[:,0]-1, mtxS[:,1]-1)), shape = shapeS).transpose()
    unspliced = sparse.csr_matrix((mtxU[:,2], (mtxU[:,0]-1, mtxU[:,1]-1)), shape = shapeU).transpose()
    ambiguous = sparse.csr_matrix((mtxA[:,2], (mtxA[:,0]-1, mtxA[:,1]-1)), shape = shapeA).transpose()

    # Load Genes and Cells identifiers
    obs = pd.read_csv(path+'Velocyto/raw/barcodes.tsv',
                  header = None, index_col = 0)

    # Remove index column name to make it compliant with the anndata format
    obs.index.name = None

    var = pd.read_csv(path+'Velocyto/raw/features.tsv', sep='\t',
                                    names = ('gene_ids', 'feature_types'), index_col = 1)
  
    # Build AnnData object to be used with ScanPy and ScVelo
    adata = anndata.AnnData(X = X, obs = obs, var = var,
                                                 layers = {'spliced': spliced, 'unspliced': unspliced, 'ambiguous': ambiguous})
    adata.var_names_make_unique()

    # Subset Cells based on STAR filtering
    selected_barcodes = pd.read_csv(path+'Gene/filtered/barcodes.tsv', header = None)
    adata = adata[selected_barcodes[0]]

    return adata.copy()

# 1. Load data and create h5 file

Load data in differents layers for velocity calculations

In [ ]:
E3_5_KO = buildAnndataFromStarCurr('E3_5_KO.out/')
E4_5_KO = buildAnndataFromStarCurr('E4_5_KO.out/')
E3_5 = buildAnndataFromStarCurr('E3_5.out/')
E4_5 = buildAnndataFromStarCurr('E4_5.out/')
E3_5_dox = buildAnndataFromStarCurr('E3_5_dox.out/')
E4_5_dox = buildAnndataFromStarCurr('E4_5_dox.out/')

In [ ]:
E3_5_KO.obs['Phenotype'] = 'E3_5_KO'
E4_5_KO.obs['Phenotype'] = 'E4_5_KO'
E3_5.obs['Phenotype'] = 'E3_5'
E4_5.obs['Phenotype'] = 'E4_5'
E3_5_dox.obs['Phenotype'] = 'E3_5_dox'
E4_5_dox.obs['Phenotype'] = 'E4_5_dox'

In [ ]:
E3_5_KO.obs['Experiment'] = 'E3_5_KO'
E4_5_KO.obs['Experiment'] = 'E4_5_KO'
E3_5.obs['Experiment'] = 'E3_5'
E4_5.obs['Experiment'] = 'E4_5'
E3_5_dox.obs['Experiment'] = 'E3_5_dox'
E4_5_dox.obs['Experiment'] = 'E4_5_dox'

In [ ]:
E3_5_KO.obs['Stage'] = 'E3_5'
E4_5_KO.obs['Stage'] = 'E4_5'
E3_5.obs['Stage'] = 'E3_5'
E4_5.obs['Stage'] = 'E4_5'
E3_5_dox.obs['Stage'] = 'E3_5'
E4_5_dox.obs['Stage'] = 'E4_5'

In [ ]:
E3_5_KO.obs['Treatment'] = 'KO'
E4_5_KO.obs['Treatment'] = 'KO'
E3_5.obs['Treatment'] = 'Control'
E4_5.obs['Treatment'] = 'Control'
E3_5_dox.obs['Treatment'] = 'dox'
E4_5_dox.obs['Treatment'] = 'dox'

In [ ]:
adata = E3_5.concatenate(E4_5,E3_5_KO,E4_5_KO,E3_5_dox,E4_5_dox, index_unique=None)

In [ ]:
adata.obs

In [ ]:
# Filter out low-represented cells and genes
sc.pp.filter_cells(adata, min_genes=150)
sc.pp.filter_genes(adata, min_cells=3)

# 2. Initial Exploratory Analysis

In [ ]:
adata.obs.Phenotype.value_counts()

In [ ]:
sns.countplot(adata.obs.Phenotype)
plt.xticks(rotation=45)

In [ ]:
sns.countplot(adata.obs.Stage)

In [ ]:
sns.countplot(adata.obs.Treatment)

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)  #save="_Fig.01.png"

# 3. Filtering

In [ ]:
# Search mitochondrial genes and annotate the group of mitochondrial genes as 'MT'
adata.var['MT'] = adata.var_names.str.startswith('mt-')
# Calculate metric
sc.pp.calculate_qc_metrics(adata, qc_vars=['MT'], percent_top=None, log1p=False, inplace=True)

In [ ]:
# Search mitochondrial genes and annotate the group of mitochondrial genes as 'MT'
adata.var['RB'] = adata.var_names.str.startswith(('Rpl','Rps'))
# Calculate metric
sc.pp.calculate_qc_metrics(adata, qc_vars=['RB'], percent_top=None, log1p=False, inplace=True)

In [ ]:
adata.obs

In [ ]:
#Plot distribution of mitochondrial genes
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_MT','pct_counts_RB'], groupby='Experiment',
             jitter=0.4, multi_panel=True, rotation=45)


In [ ]:
#Plot distribution of mitochondrial genes
sc.pl.violin(adata, ['pct_counts_MT'], groupby='Phenotype',
             jitter=0.4, multi_panel=True, rotation=45)

In [ ]:
#Plot distribution of mitochondrial genes
sc.pl.violin(adata, ['pct_counts_RB'], groupby='Phenotype',
             jitter=0.4, multi_panel=True, rotation=45)

In [ ]:
# Plot mitochondrial genes expressed
sc.pl.scatter(adata, x='total_counts', y='pct_counts_MT', size=100)
# Plot total counts
sc.pl.scatter(adata, x='total_counts', y='pct_counts_RB', size=100)
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', size=100)

In [ ]:
adata = adata[adata.obs.pct_counts_MT < 15, :]
adata = adata[adata.obs.pct_counts_RB < 25, :]

In [ ]:
adata.obs

In [ ]:
#Plot distribution of mitochondrial genes
sc.pl.violin(adata, ['pct_counts_MT'], groupby='Phenotype',
             jitter=0.4, multi_panel=True, rotation=45)

In [ ]:
#Plot distribution of mitochondrial genes
sc.pl.violin(adata, ['pct_counts_RB'], groupby='Experiment',
             jitter=0.4, multi_panel=True, rotation=45)

# 4. Normalization

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4, inplace=True)

In [ ]:
# Logarithm the data
sc.pp.log1p(adata)

In [ ]:
adata.obs

In [ ]:
# Compute highly-variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
# Plot
sc.pl.highly_variable_genes(adata)  # save="_Fig.02.png"

In [ ]:
adata.raw = adata

In [ ]:
sc.pp.scale(adata, max_value=10)

# 5. Dimensionality reduction

### PCA

In [ ]:
# Compute PCA
sc.tl.pca(adata, svd_solver='auto')

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True) 

In [ ]:
sc.pl.pca(adata, color=["Phenotype", "Stage", 'Treatment'])

### UMAP

In [ ]:
sc.pp.neighbors(adata, n_neighbors=6, n_pcs=15) #6 AND 5/!5
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=["Phenotype", "Stage", 'Treatment'])